In [1]:
import warnings
warnings.filterwarnings('ignore')


import pandas as pd
import geopandas as gpd
pd.options.display.max_columns = 999

import plotly
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot, offline
offline.init_notebook_mode(connected=True)
import plotly.express as px

from mpl_toolkits.axes_grid1 import make_axes_locatable
# %pylab inline
# pylab.rcParams['figure.figsize'] = (12, 12)
import seaborn as sns



# For changes in .py
# %reload_ext autoreload
# %autoreload 2

import numpy as np

import missingno as msno

stats = ['skew', 'mad', 'kurt']


import qgrid

def qg(df):
    return(qgrid.show_grid(df,show_toolbar=True, grid_options={'forceFitColumns': False}))

import itertools
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', -1)
pd.set_option('display.max_colwidth', 100)

import requests
import json
import xmltodict
import re
import urllib.request as urllib2
import io
import yaml

import math
import pysal as ps

# from pysal.esda.mapclassify import Quantiles, Equal_Interval, Fisher_Jenks



import imageio


from sklearn import preprocessing


In [2]:
def clean(df, ano, g_cols):
    df = df.drop(columns=['Unnamed: 16'])
    df.columns = pd.Series(df.columns).str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8').str.replace(' ','_').str.lower().str.strip().tolist()    
    mask = df['orgao'].notnull()
    df = df[mask]
    
    for col in df.columns[-6:]:
        df[col] = df[col].str.replace('.','').str.replace(',','.')
        df[col] = pd.to_numeric(df[col], errors='coerce')
    
    mask = df['dotacao_atual']!=0
    df = df[mask]
    
    values = [
                '003 - RECURSOS VINCULADOS-FUNDO ESPECIAL DE DESPES',
                '002 - RECURSOS VINCULADOS ESTADUAIS',
                '005 - RECURSOS VINCULADOS FEDERAIS',
                '045 - REC.VINC.TRANSF.FEDERAL/SUPERAVIT FINANC.',
                '007 - OP.CRED.E CONTRIB.DO EXTERIOR-DOT.INIC.CR.SU',
                '042 - REC.VINC.ESTADUAIS-CRED.SUPERAVIT FINANCEIRO'
    ]

    for value in values:
        mask = df['fonte_de_recursos']!=value
        df = df[mask]
    
    
    df['sobras'] = df['dotacao_atual'] - df['empenhado'] 
#     cols = ['dotacao_atual','empenhado','sobras']
    df = df.sort_values(by='sobras', ascending=False)
    
    
    df = df.groupby(by=g_cols, as_index=False).sum().sort_values(by='sobras', ascending=False)[g_cols + ['sobras']].rename(columns={'sobras':ano,'orgao':'orgao_{}'.format(ano)})
    
#     col = []
    
    return df 

In [3]:
ano = '2015'

df = pd.read_csv('../data/orcamento/orcamento_{}.csv'.format(ano), encoding="ISO-8859-9")

g_cols = ['orgao','fonte_de_recursos','funcao','acao']
df = clean(df,ano, g_cols)


In [4]:
years = [str(i) for i in range(2016,2020)]

# m_cols = g_cols
# m_cols.remove('orgao')

for year in years:
    df_year = pd.read_csv('../data/orcamento/orcamento_{}.csv'.format(year), encoding="ISO-8859-9")
    g_cols = ['orgao','fonte_de_recursos','funcao','acao']
    df_year = clean(df_year,year,g_cols)
    m_cols = ['fonte_de_recursos','funcao','acao']
    df = pd.merge(df,df_year,how='outer', on=m_cols)

    df = df[np.flip(np.sort(np.array(df.columns)))]


In [5]:
df = df.sort_values(by=['2018','2017','2016','2015'], ascending=False)

In [7]:
df.head(100)

,orgao_2019,orgao_2018,orgao_2017,orgao_2016,orgao_2015,funcao,fonte_de_recursos,acao,2019,2018,2017,2016,2015
3,20000 - SECRETARIA DA FAZENDA E PLANEJAMENTO,20000 - SECRETARIA DA FAZENDA E PLANEJAMENTO,20000 - SECRETARIA DA FAZENDA,20000 - SECRETARIA DA FAZENDA,20000 - SECRETARIA DA FAZENDA,09 - PREVIDENCIA SOCIAL,004 - REC.PROPRIO-ADM.IND.-DOT.INIC.CR.SUPL.,57530000 - CONCESSAO E PAGAMENTO DE BENEFICIOS,3.000881e+08,5.217329e+08,1.074179e+08,2.317924e+08,1.497371e+08
135,18000 - SECRETARIA DA SEGURANCA PUBLICA,18000 - SECRETARIA DA SEGURANCA PUBLICA,18000 - SECRETARIA DA SEGURANCA PUBLICA,18000 - SECRETARIA DA SEGURANCA PUBLICA,18000 - SECRETARIA DA SEGURANCA PUBLICA,06 - SEGURANCA PUBLICA,001 - TESOURO-DOT.INICIAL E CRED.SUPLEMENTAR,49930000 - PROTECAO E DEFESA DO CIDADAO,2.090919e+09,1.909986e+08,1.859932e+08,8.614667e+07,7.503424e+06
113,08000 - SECRETARIA DA EDUCACAO,08000 - SECRETARIA DA EDUCACAO,08000 - SECRETARIA DA EDUCACAO,08000 - SECRETARIA DA EDUCACAO,08000 - SECRETARIA DA EDUCACAO,12 - EDUCACAO,001 - TESOURO-DOT.INICIAL E CRED.SUPLEMENTAR,56960000 - CONTRIBUICAO DO ESTADO EDUCACAO BASICA-FUNDEB,1.110818e+09,1.107450e+08,1.298242e+08,1.721382e+08,9.003872e+06
93,20000 - SECRETARIA DA FAZENDA E PLANEJAMENTO,20000 - SECRETARIA DA FAZENDA E PLANEJAMENTO,20000 - SECRETARIA DA FAZENDA,20000 - SECRETARIA DA FAZENDA,20000 - SECRETARIA DA FAZENDA,04 - ADMINISTRACAO,001 - TESOURO-DOT.INICIAL E CRED.SUPLEMENTAR,50230000 - ADMINISTRACAO GERAL,7.996509e+07,1.073744e+08,4.129488e+07,3.480068e+07,1.520824e+07
117,10000 - SECR. DESENV. ECONOMICO,10000 - SECR. DESENV. ECONOMICO,"10000 - SEC.DESENV. ECONOMICO, CIENCIA,TECN. INOVACA","10000 - SEC.DESENV. ECONOMICO, CIENCIA,TECN. INOVACA","10000 - SEC.DESENV. ECONOMICO, CIENCIA,TECN. INOVACA",19 - CIENCIA E TECNOLOGIA,004 - REC.PROPRIO-ADM.IND.-DOT.INIC.CR.SUPL.,56990000 - ADMINISTRACAO DA FAPESP,4.267669e+07,9.424206e+07,1.318921e+07,1.105902e+07,8.592841e+06
1217,26000 - SECRETARIA DE INFR. MEIO AMBIENTE,39000 - SECRETARIA DE SANEAMENTO E RECURSOS HIDRICOS,NaN,NaN,NaN,18 - GESTAO AMBIENTAL,047 - REC.OPERAC. DE CREDITO-P/SUPERAVIT FINANCEIR,25340000 - IMPLANTACAO BARRAGENS DUAS PONTES E PEDREIRA,1.343137e+07,8.863120e+07,NaN,NaN,NaN
1218,39000 - SECRETARIA DE SANEAMENTO E RECURSOS HIDRICOS,39000 - SECRETARIA DE SANEAMENTO E RECURSOS HIDRICOS,NaN,NaN,NaN,18 - GESTAO AMBIENTAL,047 - REC.OPERAC. DE CREDITO-P/SUPERAVIT FINANCEIR,25340000 - IMPLANTACAO BARRAGENS DUAS PONTES E PEDREIRA,1.590000e+00,8.863120e+07,NaN,NaN,NaN
87,01000 - ASSEMBLEIA LEGISLATIVA,01000 - ASSEMBLEIA LEGISLATIVA,01000 - ASSEMBLEIA LEGISLATIVA,01000 - ASSEMBLEIA LEGISLATIVA,01000 - ASSEMBLEIA LEGISLATIVA,01 - LEGISLATIVA,001 - TESOURO-DOT.INICIAL E CRED.SUPLEMENTAR,48170000 - FUNCIONAMENTO DO PROCESSO LEGISLATIVO,4.015126e+08,8.609528e+07,5.276566e+06,4.045668e+06,1.763774e+07
1219,NaN,16000 - SECRETARIA DE LOGISTICA E TRANSPORTES,NaN,NaN,NaN,26 - TRANSPORTE,006 - OUTRAS FONTES DE RECURSOS,24970000 - NOVA TAMOIOS - CONTORNOS,NaN,8.389226e+07,NaN,NaN,NaN
4,50000 - SECRETARIA DE TURISMO,50000 - SECRETARIA DE TURISMO,50000 - SECRETARIA DE TURISMO,50000 - SECRETARIA DE TURISMO,50000 - SECRETARIA DE TURISMO,23 - COMERCIO E SERVICOS,001 - TESOURO-DOT.INICIAL E CRED.SUPLEMENTAR,41020000 - INFRAESTRUTURA TURISTICA MUNICIPIOS-ESTANCIAS,2.626127e+08,8.385191e+07,1.050450e+08,4.560729e+07,1.408174e+08


In [10]:
# df.to_excel('sobras_orcamentarias_2015_a_2019.xlsx', index=False, encoding='utf-8')

In [11]:
df

,orgao_2019,orgao_2018,orgao_2017,orgao_2016,orgao_2015,funcao,fonte_de_recursos,acao,2019,2018,2017,2016,2015
3,20000 - SECRETARIA DA FAZENDA E PLANEJAMENTO,20000 - SECRETARIA DA FAZENDA E PLANEJAMENTO,20000 - SECRETARIA DA FAZENDA,20000 - SECRETARIA DA FAZENDA,20000 - SECRETARIA DA FAZENDA,09 - PREVIDENCIA SOCIAL,004 - REC.PROPRIO-ADM.IND.-DOT.INIC.CR.SUPL.,57530000 - CONCESSAO E PAGAMENTO DE BENEFICIOS,3.000881e+08,5.217329e+08,1.074179e+08,2.317924e+08,1.497371e+08
135,18000 - SECRETARIA DA SEGURANCA PUBLICA,18000 - SECRETARIA DA SEGURANCA PUBLICA,18000 - SECRETARIA DA SEGURANCA PUBLICA,18000 - SECRETARIA DA SEGURANCA PUBLICA,18000 - SECRETARIA DA SEGURANCA PUBLICA,06 - SEGURANCA PUBLICA,001 - TESOURO-DOT.INICIAL E CRED.SUPLEMENTAR,49930000 - PROTECAO E DEFESA DO CIDADAO,2.090919e+09,1.909986e+08,1.859932e+08,8.614667e+07,7.503424e+06
113,08000 - SECRETARIA DA EDUCACAO,08000 - SECRETARIA DA EDUCACAO,08000 - SECRETARIA DA EDUCACAO,08000 - SECRETARIA DA EDUCACAO,08000 - SECRETARIA DA EDUCACAO,12 - EDUCACAO,001 - TESOURO-DOT.INICIAL E CRED.SUPLEMENTAR,56960000 - CONTRIBUICAO DO ESTADO EDUCACAO BASICA-FUNDEB,1.110818e+09,1.107450e+08,1.298242e+08,1.721382e+08,9.003872e+06
93,20000 - SECRETARIA DA FAZENDA E PLANEJAMENTO,20000 - SECRETARIA DA FAZENDA E PLANEJAMENTO,20000 - SECRETARIA DA FAZENDA,20000 - SECRETARIA DA FAZENDA,20000 - SECRETARIA DA FAZENDA,04 - ADMINISTRACAO,001 - TESOURO-DOT.INICIAL E CRED.SUPLEMENTAR,50230000 - ADMINISTRACAO GERAL,7.996509e+07,1.073744e+08,4.129488e+07,3.480068e+07,1.520824e+07
117,10000 - SECR. DESENV. ECONOMICO,10000 - SECR. DESENV. ECONOMICO,"10000 - SEC.DESENV. ECONOMICO, CIENCIA,TECN. INOVACA","10000 - SEC.DESENV. ECONOMICO, CIENCIA,TECN. INOVACA","10000 - SEC.DESENV. ECONOMICO, CIENCIA,TECN. INOVACA",19 - CIENCIA E TECNOLOGIA,004 - REC.PROPRIO-ADM.IND.-DOT.INIC.CR.SUPL.,56990000 - ADMINISTRACAO DA FAPESP,4.267669e+07,9.424206e+07,1.318921e+07,1.105902e+07,8.592841e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1552,16000 - SECRETARIA DE LOGISTICA E TRANSPORTES,NaN,NaN,NaN,NaN,26 - TRANSPORTE,044 - REC.PROP.ADM.IND-CRED.P/SUPERVAVIT FINANCEIR,60920000 - GESTAO ADMINISTRATIVA,0.000000e+00,NaN,NaN,NaN,NaN
1553,12000 - SECRETARIA DA CULTURA E ECONOMIA CRIATIVA,NaN,NaN,NaN,NaN,13 - CULTURA,043 - F.E.D - CREDITO POR SUPERAVIT FINANCEIRO,19860000 - PROAC - PROGRAMA DE ACAO CULTURAL-EDITAIS,0.000000e+00,NaN,NaN,NaN,NaN
1554,38000 - SECRETARIA ADMINISTRACAO PENITENCIARIA,NaN,NaN,NaN,NaN,14 - DIREITOS DA CIDADANIA,044 - REC.PROP.ADM.IND-CRED.P/SUPERVAVIT FINANCEIR,61430000 - FORMACAO EDUCACIONAL E PROFISSIONAL,0.000000e+00,NaN,NaN,NaN,NaN
1555,38000 - SECRETARIA ADMINISTRACAO PENITENCIARIA,NaN,NaN,NaN,NaN,14 - DIREITOS DA CIDADANIA,043 - F.E.D - CREDITO POR SUPERAVIT FINANCEIRO,61410000 - PROV.SERVICOS NECESSIDADES MATERIAIS BASICAS,0.000000e+00,NaN,NaN,NaN,NaN
